## Setup

In [1]:
!pip install openai==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
import pandas as pd
import json

In [3]:
import openai
from openai import OpenAI
client = OpenAI(
  api_key="bks-3f6fd9130cb4ba794b33bb22b1829f2fad3e0fbfc576cf78",
  base_url="https://api.trybricks.ai/api/providers/openai/v1",
)
def get_completion(prompt, model = "gpt-3.5-turbo", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
names = os.listdir('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript')[2:]
f_list = []
for n in names:
  path = '/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/' + n
  f_list.append(pd.read_csv(path))

In [21]:
map  = {list(range(len(names)))[i]: names[i] for i in range(len(names))}
map

{0: '38a5f8bc-527e-4269-b7d3-4a16d0de4e55.csv',
 1: '43e4baec-0c91-47cd-be91-50ad67d268ab.csv',
 2: '0e204b03-0e0c-49ef-aa70-52f5e4d08ff1.csv',
 3: '02c89eec-1b4f-4cb6-ae0f-ed00ae31d73f.csv',
 4: 'a7f7932a-3fa4-4797-be9c-320f61225d97.csv',
 5: 'aa642ee0-766f-42a4-bc42-9ca928ee3273.csv',
 6: '63734916-95cc-43ee-ac5c-df0268e45b4d.csv',
 7: '9b2cf845-211b-489c-b799-fac19b192aaa.csv',
 8: '1ea7c144-46fe-44e9-990b-8d81679a9a31.csv',
 9: '9225e68c-9d52-4148-952d-70f417d77c19.csv',
 10: '575ada37-caf9-4dc6-9956-6d2e115bcf9f.csv'}

## EDA

In [6]:
[len(f_list[i]) for i in range(len(f_list))] # number of turns

[387, 121, 275, 254, 73, 304, 231, 100, 73, 298]

In [7]:
[f_list[i]['Utterance end time (milliseconds)'][len(f_list[i])-1]/60000 for i in range(len(f_list))] # session length

[75.3735,
 65.7575,
 90.33033333333333,
 100.25683333333333,
 70.76933333333334,
 48.0325,
 68.21333333333334,
 42.20483333333333,
 67.67616666666666,
 59.51383333333333]

## Speaker identification

In [ ]:
idx = 9
temp = [f_list[idx]['Speaker'][i]+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [ ]:
prompt = f"""I will provide you a transcript of a tutor and a student.
There will to two speakers A and B. You task is to identify which is student and which is tutor. Make your output in a JSON format with A,B as key; student, tutor as value
""" + d
r = get_completion(prompt)

In [ ]:
a = json.loads(r)

In [ ]:
f_list[idx] = f_list[idx].replace({'Speaker': a})

In [ ]:
f_list[idx]

,Speaker,Utterance start time (milliseconds),Utterance end time (milliseconds),Utterance
0,student,8790,12710,You. That's all I wrote.
1,tutor,12790,13420,Okay.
2,student,37050,1077750,Or whatever. It.
3,C,1086120,1086870,Hello.
4,tutor,1087640,1088390,Hi.
...,...,...,...,...
109,tutor,4212020,4229590,Right. So four minus four would give us a zero...
110,C,4230680,4234150,"Well, that's good."
111,tutor,4235640,4239588,"Okay, good. That put our brains to work."
112,C,4239754,4241124,Indeed it did.


## Student expectation

In [24]:
expect = {}
for idx in range(len(f_list)):
  temp = [f_list[idx]['Speaker'][i]+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
  d = ' '.join(temp) [:4097]
  prompt = f"""I will provide you a transcript of a tutor and a student.
Find the student expectation of the session from the transcript.
Format your answer in Json format, with index of the expectation as key and content of expectation as values.
""" + d
  r = get_completion(prompt)
  result = json.loads(r)
  expect[map[idx]] = result

In [32]:
op = pd.DataFrame.from_dict(expect, orient='index')

In [34]:
op.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/student_expectations.csv', index=True)

## Off topic

In [ ]:
idx = 0
f_list[idx]['index'] = f_list[idx].index
temp = [f_list[idx]['Speaker'][i]+ "("+ str(f_list[idx]['index'][i]) + ")"+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
d = ' '.join(temp) [:4097]

In [ ]:
prompt = f"""I will provide you a transcript of a tutor and a student.
Identify the conversation that are unrelated to the learning process of the student.
Output the numbers in the parentheses
""" + d
r = get_completion(prompt)

In [ ]:
r

"Conversation unrelated to the learning process of the student:\n(13) tutor: One of these days I need to visit some folks out in Colorado. I had a couple friends in there from the field work, get them to teach me to ski. A couple of the fire guys said they'd teach me if I visited them, but I don't know. We'll see. I feel like they'd all be really harsh teachers.\n(15) tutor: I will. Arnica cream. Arnica cream. That shit helps so much. It's like, just like. It smells good. Really helps with bruising. I only know that from, like, dancing a lot. Yeah. Anyways, glad it was a good time."

## Uncertain Tone

In [40]:
Uncertain = {}
for idx in range(len(f_list)):
  f_list[idx]['index'] = f_list[idx].index
  temp = [f_list[idx]['Speaker'][i]+ "("+ str(f_list[idx]['index'][i]) + ")"+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
  d = ' '.join(temp) [:4097]
  prompt = f"""I will provide you a transcript of a tutor and a student.
  Did the tutor and student use uncertain tone during the session?
  Format your answer in Json format, with tutor and student as key and your answer as values.
  """ + d
  r = get_completion(prompt)
  result = json.loads(r)
  Uncertain[map[idx]] = result

In [42]:
op = pd.DataFrame.from_dict(Uncertain, orient='index')
op.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/uncertain_tone.csv', index=True)

## KC

In [43]:
KC = {}
for idx in range(len(f_list)):
  f_list[idx]['index'] = f_list[idx].index
  temp = [f_list[idx]['Speaker'][i]+ "("+ str(f_list[idx]['index'][i]) + ")"+ " : " + f_list[idx]['Utterance'][i]  for i in range(len(f_list[idx]))]
  d = ' '.join(temp) [:4097]
  prompt = f""" A knowledge component is a description of a mental structure or process that a learner uses, alone or in combination with other knowledge components,
 to accomplish steps in a task or a problem
I will provide you a transcript of a tutor and a student.
Identify all the acadamic knowledge components mentioned in this transcript.
Format your answer in Json format, with index of the knowledge component as key and content of knowledge component as values.
  """ + d
  r = get_completion(prompt)
  result = json.loads(r)
  KC[map[idx]] = result

In [46]:
op = pd.DataFrame.from_dict(KC, orient='index')
op.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/KC.csv', index=True)